# 專案：應用於臺北捷運時序資料之深度學習分析框架

## 筆記本 01：資料獲取與預處理

**目標：**
1.  從「臺北市資料大平臺」下載所有月份的 [捷運 OD 流量資料](https://data.taipei/dataset/detail?id=63f31c7e-7fc3-418b-bd82-b95158755b4d)。
2.  將所有分散的 CSV 檔案合併成一個單一的 DataFrame。
3.  對合併後的資料進行初步的檢視與清理。

In [1]:
import os
import sys

# 將 src 目錄添加到 Python 的模組搜索路徑中
# 這樣我們才能導入自己寫的 data_loader 模組
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
src_path = os.path.join(project_root, 'src')
if src_path not in sys.path:
    sys.path.append(src_path)

print(f"專案根目錄: {project_root}")
print(f"原始碼目錄已加入 sys.path: {src_path}")

專案根目錄: /user_data/MRT_Traffic_Analysis
原始碼目錄已加入 sys.path: /user_data/MRT_Traffic_Analysis/src


## 步驟一：下載原始資料

此步驟的目標是透過 `src/data_loader.py` 中的 `DataLoader` 類別，自動從臺北市資料大平臺抓取所有月份的 OD 流量 CSV 檔案，並儲存至 `data/raw/` 目錄下。

In [2]:
# --- 設定正確的參數 ---
# 臺北捷運每日分時各站OD流量統計資料: https://data.taipei/dataset/detail?id=63f31c7e-7fc3-418b-bd82-b95158755b4d
DATASET_ID = 'eb481f58-1238-4cff-8caa-fa7bb20cb4f4' # API 的 ID 跟 網址上的 ID 會不一樣
RAW_DATA_DIR = os.path.join(project_root, 'data', 'raw')

In [3]:
# import os
# import sys

# # --- 再次確認環境設定 ---
# project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
# src_path = os.path.join(project_root, 'src')
# if src_path not in sys.path:
#     sys.path.append(src_path)
    
# from data_loader import DataLoader

# # --- 執行下載 ---
# print("=== 開始執行最終下載程序 ===")
# loader = DataLoader(dataset_id=DATASET_ID, limit=1000)
# loader.download_monthly_data(save_dir=RAW_DATA_DIR)

# # --- 驗證下載結果 ---
# print("\n--- 下載結果驗證 ---")
# try:
#     downloaded_files = os.listdir(RAW_DATA_DIR)
#     if downloaded_files:
#         print(f"在 '{RAW_DATA_DIR}' 目錄中找到 {len(downloaded_files)} 個檔案:")
#         # 列出前5個和後5個檔案作為範例
#         sorted_files = sorted(downloaded_files)
#         print("頭5個檔案:", sorted_files[:5])
#         print("末5個檔案:", sorted_files[-5:])
#     else:
#         print("目錄為空，請檢查下載過程是否有錯誤訊息。")
# except FileNotFoundError:
#     print(f"錯誤：找不到目錄 '{RAW_DATA_DIR}'。")

## 步驟二：讀取與合併資料
將所有月份的原始 CSV 檔合併成單一 DataFrame，並進行初步檢視。

In [4]:
from data_processor import load_and_merge_raw_data
import pandas as pd

# 設定 pandas 顯示選項，以便更好地觀察 DataFrame
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

# --- 執行讀取與合併 ---
# RAW_DATA_DIR 變數我們在之前的儲存格已經定義過了
mrt_df = load_and_merge_raw_data(RAW_DATA_DIR)


# --- 初步檢視合併後的 DataFrame ---
if not mrt_df.empty:
    print("\n--- 合併後 DataFrame 的基本資訊 ---")
    mrt_df.info()
    
    print("\n\n--- 資料頭5筆 (Head) ---")
    display(mrt_df.head())
    
    print("\n\n--- 資料末5筆 (Tail) ---")
    display(mrt_df.tail())
    
    print(f"\n\n合併後的總筆數為: {len(mrt_df):,}")
else:
    print("DataFrame 是空的，請檢查前面的步驟。")

找到 28 個 CSV 檔案，準備讀取與合併...


正在讀取檔案: 100%|██████████| 28/28 [01:11<00:00,  2.55s/it]



正在合併所有資料...
所有資料已成功合併！

--- 合併後 DataFrame 的基本資訊 ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233695889 entries, 0 to 233695888
Data columns (total 5 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   日期      object
 1   時段      int64 
 2   進站      object
 3   出站      object
 4   人次      int64 
dtypes: int64(2), object(3)
memory usage: 8.7+ GB


--- 資料頭5筆 (Head) ---


,日期,時段,進站,出站,人次
0,2023-01-01,0,松山機場,松山機場,0
1,2023-01-01,0,松山機場,中山國中,0
2,2023-01-01,0,松山機場,南京復興,0
3,2023-01-01,0,松山機場,忠孝復興,0
4,2023-01-01,0,松山機場,大安,0




--- 資料末5筆 (Tail) ---


,日期,時段,進站,出站,人次
233695884,2025-04-30,23,新北產業園區,徐匯中學,0
233695885,2025-04-30,23,新北產業園區,三和國中,0
233695886,2025-04-30,23,新北產業園區,三重國小,2
233695887,2025-04-30,23,新北產業園區,迴龍,3
233695888,2025-04-30,23,新北產業園區,丹鳳,1




合併後的總筆數為: 233,695,889


## 步驟三：清理、轉換與彙總

將 8.28 億筆的原始資料進行以下處理：
1.  **建立時間戳**：將 `日期` 和 `時段` 欄位合併為標準的 `datetime` 格式。
2.  **優化資料型態**：將字串欄位轉換為 `category` 以大幅節省記憶體。
3.  **彙總資料**：將資料從「起訖站」層級，彙總到目標站點（臺北車站）的每小時進出站人次。

In [7]:
from data_processor import process_and_aggregate_data

# --- 設定分析目標 ---
# 根據上面的列表，我們使用正確的站名
TARGET_STATION = '台北車站'

# --- 執行處理與彙總 ---
# 提醒：由於資料量龐大，這一步驟會需要幾分鐘的時間
if 'mrt_df' in locals() and not mrt_df.empty:
    station_df = process_and_aggregate_data(mrt_df, target_station=TARGET_STATION)

    # --- 檢視最終彙總的 DataFrame ---
    print("\n--- 最終彙總後 DataFrame 的基本資訊 ---")
    station_df.info()

    print("\n\n--- 彙總後資料頭5筆 (Head) ---")
    display(station_df.head())

    print("\n\n--- 彙總後資料末5筆 (Tail) ---")
    display(station_df.tail())
else:
    print("錯誤：找不到名為 'mrt_df' 的 DataFrame，請確認上一步已成功執行。")

=== 開始資料清理與轉換 ===
步驟 1/4: 建立標準時間戳...
步驟 2/4: 優化記憶體使用...
--- 清理後 DataFrame 的基本資訊 ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233695889 entries, 0 to 233695888
Data columns (total 4 columns):
 #   Column  Dtype         
---  ------  -----         
 0   進站      category      
 1   出站      category      
 2   人次      int32         
 3   時間戳     datetime64[ns]
dtypes: category(2), datetime64[ns](1), int32(1)
memory usage: 3.0 GB

步驟 3/4: 彙總 '台北車站' 的進出站資料...


/user_data/MRT_Traffic_Analysis/src/data_processor.py:83: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_entries = df.groupby(['時間戳', '進站'])['人次'].sum().rename('進站人次')
/user_data/MRT_Traffic_Analysis/src/data_processor.py:85: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_exits = df.groupby(['時間戳', '出站'])['人次'].sum().rename('出站人次')


步驟 4/4: 合併 '台北車站' 的進站與出站資料...

=== '台北車站' 資料彙總完成！ ===

--- 最終彙總後 DataFrame 的基本資訊 ---
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 17880 entries, 2023-01-01 00:00:00 to 2025-04-30 23:00:00
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   進站人次    17880 non-null  int64
 1   出站人次    17880 non-null  int64
 2   總人次     17880 non-null  int64
dtypes: int64(3)
memory usage: 558.8 KB


--- 彙總後資料頭5筆 (Head) ---


,進站人次,出站人次,總人次
時間戳,,,
2023-01-01 00:00:00,1882,282,2164
2023-01-01 01:00:00,35,0,35
2023-01-01 06:00:00,941,2982,3923
2023-01-01 07:00:00,1665,3729,5394
2023-01-01 08:00:00,2619,5421,8040




--- 彙總後資料末5筆 (Tail) ---


,進站人次,出站人次,總人次
時間戳,,,
2025-04-30 19:00:00,10962,10016,20978
2025-04-30 20:00:00,7854,7106,14960
2025-04-30 21:00:00,8212,7075,15287
2025-04-30 22:00:00,8282,4411,12693
2025-04-30 23:00:00,2917,1330,4247


## 步驟四：最終定型與儲存資料
此步驟為本筆記本的最後一步。我們將對彙總後的資料進行最終處理：
1.  **填補時間間斷**：使用 `.asfreq('H')` 確保每小時都有一筆紀錄，缺失的時刻人次補 0。
2.  **儲存成果**：將處理完畢、可供分析的 DataFrame 儲存為 Pickle (`.pkl`) 檔，以便在下一本 Notebook (`02-EDA`) 中直接讀取使用。

In [8]:
import os

# --- 步驟四：最終定型與儲存資料 ---

if 'station_df' in locals() and not station_df.empty:
    
    # 1. 填補時間序列間斷，確保頻率為每小時一次
    print("--- 最終定型：填補時間序列間斷 ---")
    # 使用 asfreq('H') 確保每小時都有一個時間點，缺失的用 0 填充
    station_df_final = station_df.asfreq('H', fill_value=0)
    print(f"資料已從 {len(station_df):,} 筆擴展至 {len(station_df_final):,} 筆，確保每小時都有紀錄。")

    # 2. 定義儲存路徑 (在 data/processed/ 資料夾)
    processed_data_dir = os.path.join(project_root, 'data', 'processed')

    # 3. 如果 processed 資料夾不存在，就建立一個
    os.makedirs(processed_data_dir, exist_ok=True)
    
    # 4. 儲存為 Pickle 檔案 (推薦)
    # 檔名中標示了內容和資料區間
    final_file_path = os.path.join(processed_data_dir, 'station_timeseries_final_202301_202504.pkl')
    try:
        station_df_final.to_pickle(final_file_path)
        print(f"\n✅ 分析就緒的 DataFrame 已成功儲存至：\n   {final_file_path}")
    except Exception as e:
        print(f"❌ 儲存 Pickle 檔案失敗: {e}")

else:
    print("錯誤：找不到 'station_df' DataFrame，無法進行最終處理與儲存。")

--- 最終定型：填補時間序列間斷 ---
資料已從 17,880 筆擴展至 20,424 筆，確保每小時都有紀錄。

✅ 分析就緒的 DataFrame 已成功儲存至：
   /user_data/MRT_Traffic_Analysis/data/processed/station_timeseries_final_202301_202504.pkl


/tmp/ipykernel_9981/983296303.py:10: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  station_df_final = station_df.asfreq('H', fill_value=0)
